# Merging two codes (Nadine's code and the updated NIHAO)

## Goal

- Implement Nadine's code (two accretion models) into the most updated NIHAO simulation

### Download the codes

- Nadine's code: in dalma
- NIHAO: GitHub

### How to compare two codes easily

There were too many files to check one by one. I wrote the script to detect the difference between two corresponding files.

```Makefile.compare                    halofind.c.compare                  rotbar.c.compare
SphPressureTerms.h.compare          main.c.compare                      rotbar.h.compare
blackholes.c.compare                master.c.compare                    smooth.c.compare
cooling_metal_rad_hothalo.c.compare master.h.compare                    smooth.h.compare
cooling_metal_rad_hothalo.h.compare millerscalo.c.compare               smoothfcn.c.compare
erf.c.compare                       outtype.c.compare                   smoothfcn.h.compare
feedback.c.compare                  outtype.h.compare                   split.c.compare
feedback.h.compare                  parameters.h.compare                starform.c.compare
fio.c.compare                       pkd.c.compare                       stiff.c.compare
fof.c.compare                       pkd.h.compare                       stiff.h.compare
fof.h.compare                       pst.c.compare                       supernova.c.compare
gss.dumpframe.c.compare             pst.h.compare                       supernova.h.compare```

Then, I used the atom editor to clearly see the difference and analyze: https://atom.io/packages/split-diff

### Makefile

Don't need to change it. Just put more argument (alpha disk & torque models) if needed.

### blackhole.c

Used Nadine's code by merging it.

```#ifdef TORQUE_BH```

```#ifdef COOLING_RAD_HOTHALO```

```smFofLocalBallDisk```

```smFofRemoteBallDisk```

```smBHTor```

```pstBHTor```

```#ifdef ALPHA_BH```

```#ifdef COOLING_RAD_HOTHALO```

```smFofRemoteBallOmega```

```smFofLocalBallOmega```

```smBHAcc```

```pstBHAcc```

### SphPressureTerms.h

Since new NIHAO simulation added more chemical enrichment, I just used it as it was.

### cooling_metal_rad_hothalo.c

There were some changes.

New variables were introduced. I need to be careful not to mix them.

```  int i,j,k,l, m, n, o, p, nt, nnH, nz, nos, nns, nT6, nT7, nT8; ```

Anyway, I will use the updated version.

### cooling_metal_rad_hothalo.h

Looks not so important. Use the updated one.

### erf.c

Even though there are only three differences between two codes, there is very important thing I must keep in mind. 

```N``` in Nadine's code must be changed to ```NN```. 
I think the updated version used ```NN``` to avoid the confusion ```N``` as nitrogen. Now ```NN``` is number of something.

### feedback.c

Ok, there is some differences here. 

In the new code, there are no ```snCalcWindFeedback``` and ```snCalcUVFeedback``` functions. But it was called several times. I think these two functions are defined on another file.

Thus, I will just use the new one.

### feedback.h

Yep, ```snCalcWindFeedback``` and ```snCalcUVFeedback``` are defined in this file. Then, I can just use as it is.

### fio.c

Ok, every ```N``` was changed to ```NN```, as long as I remember ```N``` represents the number of particles (confirmed. Anyway, I can just use the new code.

### fof.c

Nothing actually has changed. Just some comments were added.

### fof.h

```void fofInitialize(FOF *pfof);```

Only this one was added to the new code. 

### gss.dumpframe.c

```    printf("filename: %s", fileout);``` 

Just this.

### halofind.c

Just added more AHF tools, and parallized the code using MPI.

### main.c

```//#ifdef CHEMPY_METALS
/*	if(msr->param.bStarForm)
		msrInitYield(msr);
                printf("number of species to track: %i",msr->param.YieldParam.nSpeciesTrack);*/
//#endif
```

```CHEMPY_METAL``` is all commented out.

### master.c

There are many differences. First, I added two accretion models part in the new code from Nadine's.

```#if defined ALPHA_BH || defined TORQUE_BH
	msr->param.nSmoothAcc = 256;
	prmAddParam(msr->prm,"nSmoothAcc",1,&msr->param.nSmoothAcc,sizeof(int),"s",
				"<number of particles to smooth accretion over> = 256");
    msr->param.dAccMax = 1.0;
	prmAddParam(msr->prm,"dAccMax", 2, &msr->param.dAccMax,
		    sizeof(double), "dAccMax",
		    "<Accretion Radius> = 1");
#endif```

```#if defined ALPHA_BH || defined TORQUE_BH
	if (!prmSpecified(msr->prm,"nSmoothAcc") )
			msr->param.nSmoothAcc = msr->param.nSmooth;
	fprintf(fp," nSmoothAcc: %i",msr->param.nSmoothAcc);
		if (!prmSpecified(msr->prm,"dAccMax") )
			msr->param.dAccMax =1;
	fprintf(fp," dAccMax: %f",msr->param.dAccMax);
#endif```

```#ifdef TORQUE_BH
	  psmf->dKpcUnit = msr->param.dKpcUnit;
		psmf->dMsolUnit = msr->param.dMsolUnit;
#endif```

```	#ifdef ALPHA_BH
	else if (iSmoothType == SMX_ALPHA || iSmoothType == SMX_BHDENSITY) {
		  in.nSmooth = msr->param.nSmooth;
		  }
#endif
	#ifdef TORQUE_BH
  else if (iSmoothType == SMX_TORQUE || iSmoothType == SMX_BHDENSITY) {
		  in.nSmooth = msr->param.nSmooth;
		  }
#endif```

```
#if defined ALPHA_BH || defined TORQUE_BH
double a = csmTime2Exp(msr->param.csm, dTime);
        if (msr->param.bBHSink) {
#ifdef ALPHA_BH
            msrBHAcc(msr, a, dTime);
#endif
#ifdef TORQUE_BH
            msrSmooth(msr, dTime, SMX_TORQUE, 1);
            msrBHTor(msr, a, dTime);
#endif
        }
    }
	    msrActiveType(msr,TYPE_GAS,TYPE_TREEACTIVE);
#ifdef COOLING_RAD_HOTHALO
	    msrBuildTree(msr,1,-1.0,1,dTime);
#else
	    msrBuildTree(msr,1,-1.0,1);  /* bTreeActive */
#endif
	    //}
#endif
    msrResetType(msr,TYPE_SINK,TYPE_SMOOTHDONE);
    msrActiveTypeRung(msr,TYPE_SINK,TYPE_ACTIVE|TYPE_SMOOTHACTIVE,iKickRung,1);
    nAccreted = msr->nGas;
    if (msr->nActive > 0) {
	msr->param.dSinkCurrentDelta = dDelta;
	msr->param.iSinkCurrentRung = iKickRung;```

I have added all those parts from Nadine's code. Analyze it and use it for developing the feedback part.

### master.h

I don't even have to add anything. Just changed ```NN```

### millerscalo.c

### outtype.c

### outtype.h

### parameters.h

Added the accretion models

```
#if defined ALPHA_BH || defined TORQUE_BH
    int nSmoothAcc;
    double dAccMax;
#endif```

### pkd.c

```double CC = Vcirc*Vcirc/(r2 + Eps*Eps);```

From ```C```, it has changed to ```CC```

### pkd.h

```#ifdef TORQUE_BH
    double accMax;
    double mdisk;
    double mgas;
    double mtot;
    double accR2;
#endif
#ifdef ALPHA_BH
    double omega;
    double accMax;
    double sigma;
    double accR2;
#endif```

New parameters are added

```#define TYPE_FORMBH            (1<<24)```

It has changed from ```1<<21```

### pst.c

```#ifdef ALPHA_BH
    mdlAddService(mdl,PST_BH_ACC,pst,
                  (void (*)(void *,void *,int,void *,int *)) pstBHAcc,
                  sizeof(struct inBH), 0);```

```#ifdef TORQUE_BH
    mdlAddService(mdl,PST_BH_TOR,pst,
                  (void (*)(void *,void *,int,void *,int *)) pstBHTor,
                  sizeof(struct inBH), 0);
#endif
#endif```

are added.

### pst.h

```#ifdef TORQUE_BH
      PST_BH_TOR,```

```#ifdef ALPHA_BH
      PST_BH_ACC,
#endif
#endif```

```
#ifdef ALPHA_BH
void pstBHAcc(PST pst,void *,int,void *,int *);
#endif
#ifdef TORQUE_BH
void pstBHTor(PST pst,void *,int,void *,int *);
#endif```

```    double dAccMax;
    double dRadiusPhys; 
    double dFac;```

### rotbar.c

```    c = rotbar->CC;```

```	intctx.c = rotbar->CC;```

```    rotbar->CC = a3 = rotbar->cratio*a2;```

### rotbar.h

```    double CC;```

### smooth.c

```#ifdef ALPHA_BH
    case SMX_ALPHA:
        smx->fcnSmooth = BHSinkOmega;
        initParticle = initBHSinkOmega; /* Original Particle */
        initTreeParticle = initTreeParticleBHSinkOmega; /* Original Particle */
        init = initBHSinkOmega; /* Cached copies */
        comb = combBHSinkOmega;
        smx->fcnPost = NULL;
        break;
#endif
#ifdef TORQUE_BH
    case SMX_TORQUE:
        smx->fcnSmooth = BHSinkDisk;
        initParticle = initBHSinkDisk; /* Original Particle */
        initTreeParticle = initTreeParticleBHSinkDisk; /* Original Particle */
        init = initBHSinkDisk; /* Cached copies */
        comb = combBHSinkDisk;
        smx->fcnPost = NULL;
        break;
#endif```

This file would be one of the most important parts I really should care about.

The function has added: ```smResmoothParticle```

Mostly I followed the new code.

### smooth.h

There is a function named ```void smGrowList(SMX smx);``` in Nadine's code. I am not sure it is necessary for the new code.

### smoothfcn.c

New, important function! Directly used to the accretion model.

```
#ifdef TORQUE_BH
void BHSinkDisk(PARTICLE *p, int nSmooth, NN *nnList, SMF *smf)
{
 
    if(TYPETest(p, TYPE_SINK)){
        double accR2=0;
        int i=0;
        for(i=0; i<nSmooth; i++){
            if (accR2< nnList[i].fDist2) accR2 = nnList[i].fDist2;
        }
        printf("a factor %f\n ", smf->a);
        if (accR2> pow((smf->dRAccMax/smf->dKpcUnit) *smf->a, 2)) accR2 = pow((smf->dRAccMax/smf->dKpcUnit) *smf->a, 2);
        p->accR2= accR2;
        printf("Radius: %f\n", sqrt(p->accR2) *smf->dKpcUnit);
}
    else{
        printf("Other Particle Present\n");}

}
```

```
void initTreeParticleBHSinkDisk(void *p)
{
}
```

```
/* Cached Tree Active particles */
void initBHSinkDisk(void *p)
{
}
```

```
void combBHSinkDisk(void *p1,void *p2)
{
}
#endif
```

```
#ifdef ALPHA_BH 
void BHSinkOmega(PARTICLE *p, int nSmooth, NN *nnList, SMF *smf)
{
    if(TYPETest(p, TYPE_SINK)){
        FLOAT mtot =0;
        FLOAT accR2=0;
        FLOAT ndark=0;FLOAT ngas =0; FLOAT nstar=0; FLOAT nsink=0;FLOAT mgas =0;
        int i=0;
        for(i=0; i<nSmooth; i++){
            if (accR2< nnList[i].fDist2) accR2 = nnList[i].fDist2;
            mtot += nnList[i].pPart->fMass;
            if(TYPETest(nnList[i].pPart, TYPE_GAS)){
                mgas += nnList[i].pPart->fMass;
                ngas++;
            }
            if(TYPETest(nnList[i].pPart, TYPE_STAR)) nstar+=1;
            if(TYPETest(nnList[i].pPart, TYPE_SINK)) nsink+=1;
            if(TYPETest(nnList[i].pPart, TYPE_DARK)) ndark+=1;
        }
        p->omega = sqrt(mtot/pow(accR2, 3/2));
        p->accR2 = accR2;
        printf("Omega= %f, mgas= %e, Omega Radius = %f , ngas= %f, nstar= %f, nsink = %f, ndark = %f \n", p->omega, mgas, sqrt(p->accR2), ngas, nstar, nsink, ndark);
        //if(TYPETest(p, TYPE_SINK)) printf("Omega Particle is a BH\n");
}
    else{
        printf("Other Particle Present\n");}
}

void initTreeParticleBHSinkOmega(void *p)
{

    }
```

```
/* Cached Tree Active particles */
void initBHSinkOmega(void *p)
{

    }
```

```
void combBHSinkOmega(void *p1,void *p2)
{
}
#endif
```

And this

```
#ifdef TORQUE_BH
	FLOAT fDisk, f0, fgas, mdisk, mtot, mgas, accR2, dYearUnit;
	FLOAT mdotTorque = 0;
#endif
#ifdef ALPHA_BH
	FLOAT mdotAlpha =0;
	FLOAT sigma, omega;
#endif```

```
#ifdef TORQUE_BH
        mgas = p->mgas;
        mdisk = p->mdisk;
        mtot = p->mtot;
        accR2 = p->accR2 /(aFac * aFac);
        
        fDisk = mdisk/mtot;
        f0 = 0.31 * fDisk*fDisk * pow(mdisk * smf->dMsolUnit / pow(10, 9), -1/3);
        fgas = mgas/mdisk;
        printf("BH Mass: %e, mDisk: %e, mgas: %e, mtot: %e, R2: %e\n",p->fMass,  mdisk, mgas, mtot, accR2);
        printf("MsolUnit %e, , Kpc Unit %e \n", smf->dMsolUnit, smf->dKpcUnit); 
        printf("BH Mass: %e x10^8, Disk Mass: %e x10^9, R0: %e kpc, fdisk: %f, f0: %f, Gas Frac: %f\n", p->fMass * smf->dMsolUnit/pow(10, 8),mdisk * smf->dMsolUnit/pow(10,9), sqrt(accR2) * smf->dKpcUnit, fDisk, f0, fgas);
        mdotTorque = pow(fDisk, 5/2)*smf->dBHSinkAlphaFactor*pow(p->fMass * smf->dMsolUnit/(pow(10, 8)), 1/6)*(mdisk * smf->dMsolUnit/(pow(10,9)))*pow(sqrt(accR2) * 10 * smf->dKpcUnit, -3/2)* pow(1 + f0/fgas, -1); 
        if (fDisk<=0 || f0<=0 || fgas <=0)mdotTorque = 0;
        //convert back to simulation units
        mdotTorque = mdotTorque / smf->dMsolUnit;
        dYearUnit = smf->dSecUnit/SECONDSPERYEAR;    
        mdotTorque = mdotTorque * dYearUnit;

    mdot = mdotTorque; /* new mdot! */
#endif 
#ifdef ALPHA_BH
       sigma = p->sigma;
       omega = p->omega;
       mdotAlpha = 3* M_1_PI * smf->dBHSinkAlphaFactor*sigma*cs*cs/(omega);
       if(omega == 0) mdotAlpha = 0;
       printf("cs:  %f, omega: %f, sigma: %f, Radius: %f \n", cs, omega, sigma, sqrt(p->accR2));
       mdot = mdotAlpha;

#endif
```

I think I changed everything I need to change.

### smoothfcn.h

Added

```#if defined ALPHA_BH || defined TORQUE_BH
    double dRAccMax;
    double dMsolUnit;
    double dKpcUnit;
#endif```

```#ifdef ALPHA_BH
  SMX_ALPHA,
#endif
#ifdef TORQUE_BH
  SMX_TORQUE,
#endif```

```#ifdef TORQUE_BH
/* SMX_ALPHA */
void BHSinkDisk(PARTICLE *p,int nSmooth,NN *nnList,SMF *smf);
void initTreeParticleBHSinkDisk(void *p);
void initBHSinkDisk(void *);
void combBHSinkDisk(void *,void *);
#endif
#ifdef ALPHA_BH
/* SMX_ALPHA */
void BHSinkOmega(PARTICLE *p,int nSmooth,NN *nnList,SMF *smf);
void initTreeParticleBHSinkOmega(void *p);
void initBHSinkOmega(void *);
void combBHSinkOmega(void *,void *);
#endif```

### split.c

```N``` to ```NN```

### starform.c

### stiff.c

a parameter ```red``` was added

### stiff.h

### supernova.c

### supernova.h

Ok done. Now compile it, and it's working